**In this project, we will clean and analyse a car sales company dataset**

This dataset has 20 columns and over 50k rows. The rows are as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.

First, we will import the Pandas e Numpy libraries, which are fundamental tools for analysing data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin_1')

In [3]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Checking the columns and some values there, we found out what we should do to make this dataset clean:

- 'price' column should have float values, but has object intead. Its name should be 'price_euros', since this is a german company dataset.
- 'odometer' column should have int values and its name should be 'kilometers'
- 'date created' could use datetime values, but has objetc instead.

In [4]:
autos = autos.rename(index=str, columns={'yearOfRegistration': 'registration_year', 'monthOfRegistration': 'registration_month', 'notRepairedDamage': 'unrepaired_damage', 'dateCreated': 'ad_created', 'dateCrawled':'date_crawled', 'offerType':'offer_type', 'vehicleType':'vehicle_type', 'powerPS':'power_ps', 'odometer': 'odometer_km', 'fuelType':'fuel_type', 'nrOfPictures': 'pictures_num', 'postalCode':'postal_code', 'lastSeen':'last_seen'})

First thing we did was correcting the columns labels, to lower case in some cases, and using underscores as separator when needed. Now we're going to make numeric those values which are in text (object) format.

Below, we convert the price column in int type, removing any non numeric character, that could get in the way.

In [5]:
price_num = autos['price'].str.replace(r'[a-zA-Z]', '')
price_num = price_num.str.replace(',','')
price_num = price_num.str.replace('$', '')
price_num = price_num.astype(int)

In [6]:
autos['price'] = price_num

In [7]:
km = autos['odometer_km'].str.replace(',','')
km = km.str.replace('km', '').astype(int)
autos['odometer_km'] = km

As we checked the price column, we've found out some aberrative values, like cars with price 0, and a car costing 9999999 EUR. That signals made clear that we should investigate further this column, and see if there are more values which do not correspond to reality.

We found other aberrative values, like cars costing 45 and 50 EUR, and others costing 1 million and more. So we've checked some car sales websites to get a general idea of minimum and maximum values found on that market.

In german, french and italian used cars websites we figured out values between 100 EUR and 100.000 EUR. We used that info to make our data cleaner, removing values out of this range.

In [8]:
valid_price = autos[autos['price'].between(100,500000)]

In [9]:
autos = valid_price
autos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48224 entries, 0 to 49999
Data columns (total 20 columns):
date_crawled          48224 non-null object
name                  48224 non-null object
seller                48224 non-null object
offer_type            48224 non-null object
price                 48224 non-null int64
abtest                48224 non-null object
vehicle_type          43801 non-null object
registration_year     48224 non-null int64
gearbox               46019 non-null object
power_ps              48224 non-null int64
model                 45829 non-null object
odometer_km           48224 non-null int64
registration_month    48224 non-null int64
fuel_type             44345 non-null object
brand                 48224 non-null object
unrepaired_damage     39340 non-null object
ad_created            48224 non-null object
pictures_num          48224 non-null int64
postal_code           48224 non-null int64
last_seen             48224 non-null object
dtypes: int64(7), objec

After cleaning entries by its prices, we kept 48.224 vehicles registers. Time to check the odometer_km column for aberrative values.

In [10]:
autos['odometer_km'].describe()

count     48224.000000
mean     125919.148142
std       39543.339640
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

Odometer column looks ok, as all its values are between 5.000km and 150.000km.

Now we will analyse columns with time values.

In [11]:
autos['date_crawled']

0        2016-03-26 17:47:46
1        2016-04-04 13:38:56
2        2016-03-26 18:57:24
3        2016-03-12 16:58:10
4        2016-04-01 14:38:50
5        2016-03-21 13:47:45
6        2016-03-20 17:55:21
7        2016-03-16 18:55:19
8        2016-03-22 16:51:34
9        2016-03-16 13:47:02
10       2016-03-15 01:41:36
11       2016-03-16 18:45:34
12       2016-03-31 19:48:22
13       2016-03-23 10:48:32
14       2016-03-23 11:50:46
15       2016-04-01 12:06:20
16       2016-03-16 14:59:02
17       2016-03-29 11:46:22
18       2016-03-26 19:57:44
19       2016-03-17 13:36:21
20       2016-03-05 19:57:31
21       2016-03-06 19:07:10
22       2016-03-28 20:50:54
23       2016-03-10 19:55:34
24       2016-04-03 11:57:02
26       2016-04-03 22:46:28
28       2016-03-19 21:56:19
29       2016-04-02 12:45:44
31       2016-03-14 16:53:09
32       2016-03-20 05:03:03
                ...         
49968    2016-04-01 17:49:15
49969    2016-03-17 18:49:02
49970    2016-03-21 22:47:37
49971    2016-

In [12]:
date_crawled = autos['date_crawled'].str[:10]

In [13]:
date_crawled.value_counts(normalize=True, dropna=False).sort_index(ascending=False).describe()

count    34.000000
mean      0.029412
std       0.009766
min       0.001389
25%       0.029881
50%       0.032743
75%       0.034905
max       0.038611
Name: date_crawled, dtype: float64

In [14]:
ad_created = autos['ad_created'].str[:10]
ad_created.value_counts(normalize=True, dropna=False).sort_index().describe()

count    76.000000
mean      0.013158
std       0.015956
min       0.000021
25%       0.000021
50%       0.000145
75%       0.032183
max       0.038860
Name: ad_created, dtype: float64

In [15]:
last_seen = autos['last_seen'].str[:10]
last_seen.value_counts(normalize=True, dropna=False).sort_index().describe()

count    34.000000
mean      0.029412
std       0.043780
min       0.001078
25%       0.012457
50%       0.019430
75%       0.023816
max       0.221964
Name: last_seen, dtype: float64

In [16]:
print ('Top days for date crawled, date of ad creation and last seen')
print (date_crawled.max())
print (ad_created.max())
print (last_seen.max())

Top days for date crawled, date of ad creation and last seen
2016-04-07
2016-04-07
2016-04-07


In [17]:
print ('The worst day for the same')
print (date_crawled.min())
print (ad_created.min())
print (last_seen.min())

The worst day for the same
2016-03-05
2015-06-11
2016-03-05


In [18]:
registration_year = autos['registration_year']
registration_year.describe()

count    48224.000000
mean      2004.730964
std         87.897388
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

As we can see, there are entries with registration year equal to 9999 and before 1000, which is suspect, since cars didnt exist until 1900 and the dataset was compiled in 2016. We checked how many entries we would lose eliminating those.

In [19]:
safe_reg_year_bool = autos['registration_year'].between(1900, 2016)

In [20]:
safe_years_autos = autos[safe_reg_year_bool]
safe_years_autos

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,pictures_num,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50
5,2016-03-21 13:47:45,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,privat,Angebot,7900,test,bus,2006,automatik,150,voyager,150000,4,diesel,chrysler,NaN,2016-03-21 00:00:00,0,22962,2016-04-06 09:45:21
6,2016-03-20 17:55:21,VW_Golf_III_GT_Special_Electronic_Green_Metall...,privat,Angebot,300,test,limousine,1995,manuell,90,golf,150000,8,benzin,volkswagen,NaN,2016-03-20 00:00:00,0,31535,2016-03-23 02:48:59
7,2016-03-16 18:55:19,Golf_IV_1.9_TDI_90PS,privat,Angebot,1990,control,limousine,1998,manuell,90,golf,150000,12,diesel,volkswagen,nein,2016-03-16 00:00:00,0,53474,2016-04-07 03:17:32
8,2016-03-22 16:51:34,Seat_Arosa,privat,Angebot,250,test,NaN,2000,manuell,0,arosa,150000,10,NaN,seat,nein,2016-03-22 00:00:00,0,7426,2016-03-26 18:18:10
9,2016-03-16 13:47:02,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,privat,Angebot,590,control,bus,1997,manuell,90,megane,150000,7,benzin,renault,nein,2016-03-16 00:00:00,0,15749,2016-04-06 10:46:35


After eliminating cars with registration years out of the 1900-2016 range, we still have 46352 entries, which remains a good number to our work. Now, let's clean the original dataset from these values and check the distribution of values by year.

In [21]:
autos = safe_years_autos
autos['registration_year'].value_counts(normalize=True)

2000    0.066966
2005    0.062802
1999    0.062112
2004    0.058228
2003    0.058099
2006    0.057560
2001    0.056718
2002    0.053439
1998    0.050483
2007    0.049038
2008    0.047679
2009    0.044874
1997    0.041530
2011    0.034907
2010    0.034238
1996    0.029233
2012    0.028219
2016    0.025932
1995    0.025738
2013    0.017281
2014    0.014282
1994    0.013505
1993    0.009061
2015    0.008198
1992    0.007918
1991    0.007292
1990    0.007163
1989    0.003689
1988    0.002869
1985    0.002006
          ...   
1966    0.000475
1976    0.000453
1969    0.000410
1975    0.000388
1965    0.000367
1964    0.000259
1963    0.000173
1961    0.000129
1959    0.000129
1962    0.000086
1956    0.000086
1937    0.000086
1958    0.000086
1955    0.000043
1954    0.000043
1957    0.000043
1951    0.000043
1910    0.000043
1934    0.000043
1941    0.000043
1927    0.000022
1929    0.000022
1950    0.000022
1931    0.000022
1948    0.000022
1938    0.000022
1939    0.000022
1953    0.0000

We found out that the year with most cars registered on the dataset is 2000, with 6,6% of the total. Most of the vehicles on the dataset were registered between 1997 and 2016, with percentages around 2% and 6%.

Now, we will find the most expensive and the cheapest brands (by mean), and find the most expensive and the most cheap cars.

In [22]:
expensive_cars = {}
cheap_cars = {}
top_car = {}
top_mean_price = {}

unibrand = autos['brand'].unique()

for brand in unibrand:
    selected_rows = autos[autos['brand'] == brand]
        
    expensive = selected_rows.sort_values('price', ascending=False)
    cheap = selected_rows.sort_values('price', ascending=True)
        
    top = expensive[:1]
    top = top[['name', 'price', 'registration_year']]
    top_car[brand] = top
    
    expensive_mean = expensive['price'].mean()
    top_mean_price[brand] = expensive_mean
    
    cheapest = cheap[:1]
    cheapest = cheap[['name', 'price', 'registration_year']]
    

In [23]:
top_mean = sorted(top_mean_price.items(), key=lambda kv: kv[1], reverse=True)

In [24]:
top_mean

[('porsche', 46788.444444444445),
 ('land_rover', 19108.091836734693),
 ('sonstige_autos', 12784.556561085972),
 ('jaguar', 11961.56338028169),
 ('jeep', 11650.5),
 ('mini', 10639.450980392157),
 ('audi', 9380.718547986076),
 ('mercedes_benz', 8672.654241071428),
 ('bmw', 8381.677305658899),
 ('chevrolet', 6759.885931558935),
 ('skoda', 6409.609724047306),
 ('kia', 6018.442073170731),
 ('dacia', 5915.528455284553),
 ('volkswagen', 5436.950096948668),
 ('hyundai', 5411.075431034483),
 ('toyota', 5167.091062394604),
 ('volvo', 4993.208037825059),
 ('nissan', 4756.659634317863),
 ('seat', 4433.419621749409),
 ('mazda', 4129.774787535411),
 ('suzuki', 4126.341818181818),
 ('honda', 4119.109589041096),
 ('alfa_romeo', 4100.915857605178),
 ('subaru', 4033.7551020408164),
 ('citroen', 3796.26267281106),
 ('ford', 3779.269076305221),
 ('smart', 3596.40273556231),
 ('chrysler', 3486.5766871165642),
 ('lancia', 3444.877551020408),
 ('mitsubishi', 3439.10290237467),
 ('saab', 3211.6493506493507),

Here is the cheapest vehicle on the dateset, a Lancia Bastler 1999.

In [35]:
cheapest[:1]

,name,price,registration_year
9962,bastler_auto,195,1999


And here is the top expensive car, the Porsche 991 2016.

In [26]:
top_car['porsche']

,name,price,registration_year
36818,Porsche_991,350000,2016


We will now compare kilometrage and price by brand, to see if there's some relation between these two metrics.

Below, we created a dictionary with the mean kilometrage by brand.

In [27]:
mean_kilometrage = {}

for brand in unibrand:
    selected_rows = autos[autos['brand'] == brand]
    meankm = selected_rows['odometer_km'].mean()
    mean_kilometrage[brand] = meankm
    

In [28]:
mean_km_apres = sorted(mean_kilometrage.items(), key=lambda kv: kv[1], reverse=True)

And now, we will organize the list, by the top kilometrage by brand and present it.

In [36]:
mean_km_apres

[('saab', 144415.58441558442),
 ('volvo', 138581.56028368796),
 ('rover', 137661.29032258064),
 ('chrysler', 133159.50920245398),
 ('bmw', 132695.32014881534),
 ('alfa_romeo', 131747.572815534),
 ('mercedes_benz', 131025.66964285714),
 ('opel', 129384.42969221485),
 ('audi', 129245.40029835903),
 ('volkswagen', 128799.87753852434),
 ('renault', 128281.3932172319),
 ('peugeot', 127127.8901734104),
 ('jeep', 127122.64150943396),
 ('mitsubishi', 127097.6253298153),
 ('subaru', 125612.24489795919),
 ('jaguar', 125422.5352112676),
 ('mazda', 124553.82436260623),
 ('ford', 124277.10843373495),
 ('honda', 122493.1506849315),
 ('lancia', 122346.9387755102),
 ('seat', 121536.64302600473),
 ('daewoo', 121231.88405797101),
 ('citroen', 119554.53149001536),
 ('nissan', 118326.30098452883),
 ('land_rover', 118010.20408163265),
 ('fiat', 116950.29486099411),
 ('daihatsu', 116120.68965517242),
 ('toyota', 115944.35075885328),
 ('kia', 112301.82926829268),
 ('skoda', 110906.70170827858),
 ('suzuki', 1

In order to extract these valus, we had to create dictionaries, which are not an ideal final format for working with data. We will transforms these dictionaries into a single dataframe with the coding below.

In [37]:
mean_prices = pd.Series(top_mean_price).astype(float)
mean_km = pd.Series(mean_kilometrage)

In [38]:
km_price_mean = pd.DataFrame()
km_price_mean['mean_price'] = mean_prices

In [51]:
km_price_mean['mean_km'] = mean_km

Let's take a look at the new Dataframe we have.

In [52]:
km_price_mean

,mean_price,mean_km
alfa_romeo,4100.915858,131747.572816
audi,9380.718548,129245.400298
bmw,8381.677306,132695.320149
chevrolet,6759.885932,100266.159696
chrysler,3486.576687,133159.509202
citroen,3796.262673,119554.531490
dacia,5915.528455,84268.292683
daewoo,1064.057971,121231.884058
daihatsu,1649.655172,116120.689655
fiat,2836.873631,116950.294861


Now we have a new dataset with the two columns, it will be easier to make operations between the two columsn. So, we will create a index, dividing the mean brand kilometrage by the mean brand price.

Our goal is to find out how many kilometers had the brands ran in comparison to the their value in Euros.

In [50]:
index_km_value = km_price_mean['mean_km'] / km_price_mean['mean_price']

And here's the result:

In [47]:
index_km_value.sort_values()

porsche             2.104723
land_rover          6.175928
sonstige_autos      7.070713
mini                8.300130
jaguar             10.485463
jeep               10.911346
audi               13.777772
dacia              14.245269
chevrolet          14.832522
mercedes_benz      15.107909
bmw                15.831595
skoda              17.303191
kia                18.659618
hyundai            19.753148
toyota             22.438999
volkswagen         23.689730
nissan             24.875923
suzuki             26.212961
seat               27.413747
smart              27.731611
volvo              27.754013
honda              29.737774
mazda              30.159956
trabant            30.387429
lada               31.067453
subaru             31.140275
citroen            31.492692
alfa_romeo         32.126378
ford               32.883901
lancia             35.515613
mitsubishi         36.956622
chrysler           38.192049
peugeot            40.826456
fiat               41.225063
opel          

We found out that the most expensive brand are in fact the less used ones, the ones with lowest kilometrage.

Now, we will find the most commom brand/model combination, and the least common.

In [85]:
brand_ocurrences = {}
brand_total_vehicles = {}

for brand in unibrand:
    selected_rows = autos[autos['brand'] == brand]
    brand_count = selected_rows['model']
    brand_ocurrences[brand] = brand_count
    brand_total_vehicles[brand] = brand_ocurrences[brand].shape[0]

In [86]:
brand_total_vehicles_org = sorted(brand_total_vehicles.items(), key=lambda kv: kv[1], reverse=True)

This is the list of the most commom brands on the dataset. Volkswagen, BMW, Opel, Mercedes-Benz and Audi are the most common ones. It makes sense, since this is a german dataset, and all the brands with most vehicles for sale are german too.

In [87]:
brand_total_vehicles_org

[('volkswagen', 9799),
 ('bmw', 5107),
 ('opel', 4971),
 ('mercedes_benz', 4480),
 ('audi', 4022),
 ('ford', 3237),
 ('renault', 2182),
 ('peugeot', 1384),
 ('fiat', 1187),
 ('seat', 846),
 ('skoda', 761),
 ('nissan', 711),
 ('mazda', 706),
 ('smart', 658),
 ('citroen', 651),
 ('toyota', 593),
 ('hyundai', 464),
 ('sonstige_autos', 442),
 ('volvo', 423),
 ('mini', 408),
 ('mitsubishi', 379),
 ('honda', 365),
 ('kia', 328),
 ('alfa_romeo', 309),
 ('porsche', 279),
 ('suzuki', 275),
 ('chevrolet', 263),
 ('chrysler', 163),
 ('dacia', 123),
 ('daihatsu', 116),
 ('jeep', 106),
 ('subaru', 98),
 ('land_rover', 98),
 ('saab', 77),
 ('jaguar', 71),
 ('daewoo', 69),
 ('trabant', 63),
 ('rover', 62),
 ('lancia', 49),
 ('lada', 27)]

In [98]:
volks_models = autos[autos['brand'] == 'volkswagen']

As we can see, Volkswagen is the most common brand. And now, the most common model.

In [99]:
volks_models['model'].value_counts()

golf           3684
polo           1592
passat         1345
transporter     671
touran          433
lupo            320
sharan          218
caddy           203
beetle          123
tiguan          118
bora            100
andere           95
touareg          94
scirocco         85
fox              82
eos              66
kaefer           56
up               51
jetta            38
phaeton          31
cc               18
amarok            6
Name: model, dtype: int64

And we conclude that Volkswagen Golf is the most common Brand/Model on this dataset.

We will now check the value difference on vehicles with unrepaired damage.

In [103]:
damage = autos[autos['unrepaired_damage'] == 'ja']

In [110]:
no_damage = autos[autos['unrepaired_damage'] != 'ja']

As we can see, vehicles with unrepaired damage has around 3 times less values than others.

In [108]:
no_damage['price'].mean()

6421.396393599236

In [109]:
damage['price'].mean()

2269.5798750557788